In [1]:
import scipy.io as sio

In [8]:
mat_data = sio.loadmat(r'C:\Users\medici\901\학습용 데이터1\내륜결함\1.mat')

In [9]:
mat_data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Oct  8 18:13:58 2024',
 '__version__': '1.0',
 '__globals__': [],
 'fs': array([[25600]], dtype=uint16),
 'signal': array([[-0.20226767],
        [-0.35395942],
        [-0.11909487],
        ...,
        [ 0.00184211],
        [-0.21885705],
        [ 0.00095067]])}

In [10]:
mat_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'fs', 'signal'])

In [12]:
len(mat_data.items())

5

In [13]:
signal_data = mat_data['signal']

In [15]:
print(signal_data)

[[-0.20226767]
 [-0.35395942]
 [-0.11909487]
 ...
 [ 0.00184211]
 [-0.21885705]
 [ 0.00095067]]


In [16]:
print(signal_data.shape)

(25600, 1)


In [17]:
mat2_data = sio.loadmat(r'C:\Users\medici\901\학습용 데이터1\내륜결함\2.mat')

In [18]:
signal2_data = mat2_data['signal']

In [19]:
print(signal2_data.shape)

(25600, 1)
